# My First Deep Learning Sheet

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### import data

In [ ]:
df = pd.read_csv("../input/heart-disease-uci/heart.csv")

let's check raw data

In [ ]:
df.info()
df.describe()

In [ ]:
df.head()

#### As you can see my labels are male and female , first i need to split my features and label then encode 

In [ ]:
y = df.sex.values
x = df.drop(["sex"], axis=1)

In [ ]:
y


In [ ]:
x.head()

##### we make normalize

In [ ]:
x = (x-np.min(x))/(np.max(x)-np.min(x)).values

Let's check  our data 

In [ ]:
x.head()

### I will split data for x_training,x_testing and y_training,y_testing

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

x_train = x_train.T
x_test = x_test.T
y_train = y_train.T
y_test = y_test.T

In [ ]:
x_train.head()
x_train.shape

First we need initialize weight,bias and sigmoid function

In [ ]:
def weight_bias(dimension):            # our dimension is 13
    w = np.full((dimension,1),0.01)
    b = 0.0
    return w,b

# sigmoid function is  1/(1+e^(-x))

def sigmoid(z):
    y_head = 1/(1+np.exp(-z))
    return y_head

In [ ]:
sigmoid(0)

In [ ]:
# i will write forward and backward propagation

def forward_backward_propagation(w,b,x_train,y_train):
    # forward
    
    z = np.dot(w.T,x_train)+b
    y_head = sigmoid(z)
    loss = -y_train*np.log(y_head)-(1-y_train)*np.log(1-y_head)
    cost = (np.sum(loss))/x_train.shape[1]   
    # backward
    
    derivative_weight = (np.dot(x_train,((y_head-y_train).T)))/x_train.shape[1] # x_train.shape[1]  is for scaling
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]                 # x_train.shape[1]  is for scaling
    gradients = {"derivative_weight": derivative_weight,"derivative_bias": derivative_bias}
    return cost,gradients

In [ ]:
# updating
def update(w,b,learning_rate,iteration):
    cost_list = []
    cost_list2 = []
    index = []
    #learning
    
    for i in range(iteration):
        cost,gradients = forward_backward_propagation(w,b,x_train,y_train)
        cost_list.append(cost)
        
        w = w - learning_rate*gradients["derivative_weight"]
        b = b - learning_rate*gradients["derivative_bias"]
        if i%10 == 0: 
            cost_list2.append(cost)
            index.append(i)
            print ("Cost after iteration %i: %f" %(i, cost))
    # visualization
    parameters = {"weight": w,"bias": b}
    plt.plot(index,cost_list2)
    plt.xlabel("Number of Iterarion")
    plt.ylabel("Cost")
    plt.show()
    return parameters, cost_list, gradients
#parameters, gradients, cost_list = update(w, b, x_train, y_train, learning_rate = 0.009,number_of_iterarion = 200)

In [ ]:
# prediction

def prediction(w,b,x_test):
    #parameters, gradients, cost_list = update(w, b, x_train, y_train, learning_rate = 0.009,number_of_iterarion = 200)
    z = sigmoid(np.dot(w.T,x_test)+b)
    Y_prediction = np.zeros((1,x_test.shape[1]))
    
    for i in range(x_test.shape[1]):
        if z[0,i]<= 0.5:
              Y_prediction[0,i] = 0
        else:
              Y_prediction[0,i] = 1
            
    return Y_prediction

In [ ]:
# finally

def logical_regression(x_train,x_test,y_train,y_test, learning_rate,iteration):
    dimension =x_train.shape[0]
    w,b = weight_bias(dimension)
    
    parameters, cost_list, gradient = update(w,b,learning_rate,iteration)
    
    y_prediction = prediction(parameters["weight"],parameters["bias"],x_test)
    print("test accuracy: {} %".format(100 - np.mean(np.abs(y_prediction - y_test)) * 100))

Let's try it

In [ ]:
logical_regression(x_train, x_test, y_train, y_test, learning_rate = 1, iteration = 150)

In [ ]:
logical_regression(x_train, x_test, y_train, y_test, learning_rate = 0.1, iteration = 150)

In [ ]:
logical_regression(x_train, x_test, y_train, y_test, learning_rate = 0.01, iteration = 150)